This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [26]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv('.env')
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

In [12]:
!ls

api_data_wrangling_mini_project.ipynb


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [13]:
# First, import the relevant modules
import requests


Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [16]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?limit=1&api_key='+ API_KEY
r = requests.get(url)


In [17]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(r.json())

{'dataset_data': {'limit': 1, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2000-06-07', 'end_date': '2020-12-01', 'frequency': 'daily', 'data': [['2020-12-01', 112.2, 112.2, 111.5, 112.0, None, 51.0, 5703.0, None, None, None]], 'collapse': None, 'order': None}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [18]:
#1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key='+API_KEY
r = requests.get(url)


In [20]:
#2. Convert the returned JSON object into a Python dictionary.
json_data = r.json()


In [21]:
#3. Calculate what the highest and lowest opening prices were for the stock in this period.
open_prices = [daily[1] for daily in json_data['dataset_data']['data'] if daily[1] != None]
max_open = str(max(open_prices))
min_open = str(min(open_prices))
print('The highest opening price was $' + max_open + ' and the lowest opening price was $' + min_open + ' in 2017.')

The highest opening price was $53.11 and the lowest opening price was $34.0 in 2017.


In [22]:
#4. What was the largest change in any one day (based on High and Low price)?
daily_change = [daily[2]-daily[3] for daily in json_data['dataset_data']['data'] if daily[2:3] != None]
largest_change = str(round(max(daily_change), 2))
print('The largest change in any one day was $' + largest_change + ' in 2017.')

The largest change in any one day was $2.81 in 2017.


In [28]:
# 5. What was the largest change between any two days (based on Closing Price)?
close_prices = [daily[4] for daily in json_data['dataset_data']['data'] if daily[4] != None]
change = [x - close_prices[i-1] for i,x in enumerate(close_prices)][1:]
pos_change = [abs(num) for num in change]
largest_close_change = str(round(max(pos_change), 2))
print('The largest change between any two consecutive days based on Closing Price was $' + largest_close_change + ' in 2017.')

The largest change between any two consecutive days based on Closing Price was $2.56 in 2017.


In [24]:
#6. What was the average daily trading volume during this year?
daily_trading_volume = [daily[6] for daily in json_data['dataset_data']['data'] if daily[6] != None]
average = str(round(sum(daily_trading_volume)/len(daily_trading_volume),1))
print('The average daily trading volume during 2017 was ' + average + '.')

The average daily trading volume during 2017 was 89124.3.


In [25]:
#7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)
daily_trading_volume = [daily[6] for daily in json_data['dataset_data']['data'] if daily[6] != None]
def median(anylist):
    sorted_anylist = sorted(anylist)
    len_anylist = len(anylist)
    index = (len_anylist-1)//2
    
    if (len_anylist%2):
        return sorted_anylist[index]
    else:
        return (sorted_anylist[index]+sorted_anylist[index+1])//2.0
median = str(median(daily_trading_volume))
print('The median trading volume during 2017 was ' + median + '.')

The median trading volume during 2017 was 76286.0.
